In [21]:
import cv2
import numpy as np
# 1. Use OpenCV’s VideoCapture to load the input video.

input_video_path = 'squat_18.mp4'
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# 2. Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS
# You’ll need it to calculate the timestamp for each frame.
# cv2_CAP_PROP_FPS is represented by the integer 5 in modern OpenCV (??)

fps = cap.get(cv2.CAP_PROP_FPS)
# Check if fps is greater than zero before calculating frame_interval_ms
if fps > 0:
    frame_interval_ms = 1000 / fps      #Time between frames in milliseconds (ms)
else:
    print("Error: Could not get video frame rate.")
    exit()

In [22]:
!python -m pip install mediapipe

In [23]:
!pip install mediapipe opencv-python

In [24]:
model_path = 'pose_landmarker_full.task'

In [25]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [26]:
# Initialize the drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [27]:
# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(base_options=BaseOptions(model_asset_path=model_path),running_mode=VisionRunningMode.VIDEO)
frame_number = 0
current_timestamp_ms = 0 # Initialize timestamp

with PoseLandmarker.create_from_options(options) as landmarker:
    # 3. Loop through each frame in the video using VideoCapture.read()
    while cap.isOpened():
        ret, frame = cap.read() # ret is a boolean, frame is the image array

        if not ret:
            # End of video stream or error
            break

        # Calculate the timestamp for the current frame
        # MediaPipe requires the time since the start of the video in milliseconds (ms)
        current_timestamp_ms = int(frame_number * frame_interval_ms)

        # 4. Convert the frame received from OpenCV to a MediaPipe’s Image object.
        # where OpenCV reads frames in BGR format; MediaPipe expects RGB, Convert BGR (OpenCV) to RGB (MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

        # 5. Run the detection on the frame
        pose_result = landmarker.detect_for_video(mp_image, current_timestamp_ms)

        # Check if any person was detected in the frame
        if pose_result.pose_landmarks:

            # Print general frame info
            print(f"\n--- Frame {frame_number} ---")

            # We will only process the first detected person (index 0) for brevity.
            person_landmarks = pose_result.pose_landmarks[0]
            person_world_landmarks = pose_result.pose_world_landmarks[0]

            print(f"  Person 1 detected with 33 landmarks:")

            # Iterate through all 33 landmarks
            # 'landmark_enum' provides the name (e.g., 'NOSE')
            for landmark_enum in mp.solutions.pose.PoseLandmark:
                landmark_index = landmark_enum.value
                landmark_name = landmark_enum.name

                # Get the normalized coordinates
                norm_point = person_landmarks[landmark_index]

                # Get the 3D world coordinates
                world_point = person_world_landmarks[landmark_index]

                # Print the data
                print(f"    {landmark_name} ({landmark_index}):")
                print(f"      World (X, Y, Z, V): {world_point.x:.4f}m, {world_point.y:.4f}m, {world_point.z:.4f}m, Vis: {norm_point.visibility:.2f}")

        # Else, print no pose detected message
        else:
            print(f"Frame {frame_number}: No pose detected.")

        # Increment frame count
        frame_number += 1

Streaming output truncated to the last 5000 lines.
    LEFT_PINKY (17):
      World (X, Y, Z, V): 0.2181m, -0.4545m, -0.5687m, Vis: 0.97
    RIGHT_PINKY (18):
      World (X, Y, Z, V): -0.1949m, -0.3747m, -0.6914m, Vis: 0.94
    LEFT_INDEX (19):
      World (X, Y, Z, V): 0.1998m, -0.4650m, -0.5621m, Vis: 0.97
    RIGHT_INDEX (20):
      World (X, Y, Z, V): -0.1688m, -0.3822m, -0.6776m, Vis: 0.94
    LEFT_THUMB (21):
      World (X, Y, Z, V): 0.2057m, -0.4505m, -0.5321m, Vis: 0.95
    RIGHT_THUMB (22):
      World (X, Y, Z, V): -0.1859m, -0.3726m, -0.6464m, Vis: 0.92
    LEFT_HIP (23):
      World (X, Y, Z, V): 0.1184m, 0.0007m, 0.0102m, Vis: 1.00
    RIGHT_HIP (24):
      World (X, Y, Z, V): -0.1178m, -0.0008m, -0.0092m, Vis: 1.00
    LEFT_KNEE (25):
      World (X, Y, Z, V): 0.2110m, 0.2356m, -0.2674m, Vis: 1.00
    RIGHT_KNEE (26):
      World (X, Y, Z, V): -0.1917m, 0.2194m, -0.2730m, Vis: 1.00
    LEFT_ANKLE (27):
      World (X, Y, Z, V): 0.2255m, 0.5618m, -0.2513m, Vis: 0.99
    

In [29]:
import time
# Imports specific for Google Colab visualization
from google.colab.patches import cv2_imshow
from IPython.display import clear_output


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    cap = cv2.VideoCapture(input_video_path)
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("\nEnd of video stream.")
            break

        # 1. Prepare Frame for MediaPipe
        current_timestamp_ms = int(frame_number * frame_interval_ms)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 2. Run Detection
        pose_result = pose.process(rgb_frame)

        # 3. Prepare Frame for Visualization (Convert RGB back to BGR)
        annotated_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # 4. Process Results and Visualize
        if pose_result.pose_landmarks:

            # Process the first detected person (index 0)
            # pose_result.pose_landmarks is a list of NormalizedLandmarkList,
            # each NormalizedLandmarkList contains the landmarks for a detected person.
            # The drawing utilities expect a NormalizedLandmarkList, not a list of them.
            person_landmarks = pose_result.pose_landmarks # Pass the entire NormalizedLandmarkList

            # Draw the skeleton onto the frame
            mp_drawing.draw_landmarks(
                annotated_frame,
                person_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

        # 5. Display Frame in Colab
        # display every 5 frames to prevent output lag/crashes in Colab
        if frame_number % 5 == 0:

            display_frame = cv2.resize(annotated_frame, (640, 480))

            # Use the Colab-specific imshow
            cv2_imshow(display_frame)

            # Clear output for "real-time" update effect, then pause
            clear_output(wait=True)
            time.sleep(1 / fps)

        # 6. Advance Frame Counter
        frame_number += 1

# --- CLEANUP ---
cap.release()
cv2.destroyAllWindows()
print("Visualization and processing finished.")


End of video stream.
Visualization and processing finished.
